In [ ]:
import sqlalchemy as sq
import pymysql

import pandas as pd
import numpy as np

import os
import csv

import datetime as dt

sq.__version__

In [ ]:

def convert_unixtime(stamp):
    return dt.datetime.fromtimestamp(
        int(stamp)
    ).strftime('%Y-%m-%d')

In [ ]:
# Set hash function to anonymize data
import hashlib

SALT = os.urandom(16)

def anonymize(value):
    return hashlib.sha1(value+SALT).hexdigest()

In [ ]:
def find_dept(email):
    ampersand = email.find('@')
    tail = email[ampersand + 1:]
    try:
        return dept_dict[tail]
    except KeyError:
        return "OTHER"

## Set up Dept List/Dict

In [ ]:
dept_dict = {}

data_path = r'/Users/toferc/Documents/Data/'
output_path = r'/Users/toferc/Documents/Report Card/April/'

In [ ]:
with open(os.path.join(data_path, 'csv_keys.csv'), "r") as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    
    for row in reader:
        email, acronym = row
        dept_dict[email] = acronym

dept_dict['cadets.gc.ca'] = 'CADETS'
dept_dict['canada.gc.ca'] = 'CANADA'
dept_dict['canada.ca'] = 'CANADA'
dept_dict['tribunal.gc.ca'] = 'TRIBUNAL'
dept_dict['cannor.gc.ca'] = 'CED/DEC'
dept_dict['ci-oic.gc.ca'] = 'CI/OIC'
dept_dict['ccgs-ngcc.gc.ca'] = 'CCGS/NGCC'
dept_dict['god.ccgs-ngcc.gc.ca'] = 'CCGS/NGCC'
dept_dict['clo-ocol.gc.ca'] = 'OCOL/CLO'
dept_dict['csps.gc.ca'] = 'CSPS/EFPC'
dept_dict['interenational.gc.ca'] = 'DFAITD/MAECD'
dept_dict['cnb-ncw.gc.ca'] = 'CNB/NCW'
dept_dict['ncw-cnb.gc.ca'] = 'CNB/NCW'
dept_dict['nfb.gc.ca'] = 'NFB/ONF'
dept_dict['nrccan-rncan.gc.ca'] = 'NRCAN/RNCAN'
dept_dict['nserc-crsng.gc.ca'] = 'NSERC/CRSNG'
dept_dict['pbc-clcc.gc.ca'] = 'PBC/CLCC'
dept_dict['pco.bcp.gc.ca'] = 'PCO/BCP'
dept_dict['pipsc.ca'] = 'PIPSC/IPFPC'
dept_dict['ps.sp.gc.ca'] = 'PS/SP'
dept_dict['servicecanada.gc.ca.gc.ca'] = 'HRSDC/RHDSC'
dept_dict['fintrac-canafe.gc.ca'] = 'FINTRAC'
dept_dict['gmail.com'] = 'GMAIL'
dept_dict['tribunbal.gc.ca'] = 'TRIBUNAL'

In [ ]:
dept_list = []

for k, v in dept_dict.items():
    dept_list.append(v)

dept_list = set(dept_list)

In [ ]:
# Load CIOC members and FTE counts

cioc_depts = pd.read_csv(os.path.join(data_path, 'CIOC_depts_jan_2016.csv'),
                        thousands=',')

In [ ]:
cioc_depts.head()

## Connect to DB

In [ ]:
import getpass

password = getpass.getpass('Enter Password: ')

In [ ]:
# MariahDB = 165
# MYSQL = 117

db_connection = "mysql+pymysql://elgg:{}@192.168.1.99:3306/elgg12".format(
    password)

In [ ]:
engine = sq.create_engine(db_connection,encoding='latin1', echo=True)

In [ ]:
conn = engine.connect()

In [ ]:
engine.connect()

In [ ]:
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import and_, or_
Session = sessionmaker(bind=engine)

In [ ]:
Session.configure(bind=engine)
session = Session()

In [ ]:
Base = automap_base()

Base.prepare(engine, reflect=True)

In [ ]:
# Set up mappings

Users = Base.classes.elggusers_entity
Groups = Base.classes.elgggroups_entity
Relationships = Base.classes.elggentity_relationships
Entities = Base.classes.elggentities
Objects = Base.classes.elggobjects_entity
MetaData = Base.classes.elggmetadata
MetaStrings = Base.classes.elggmetastrings
Annotations = Base.classes.elggannotations


### Guide to Elgg Entities

Blogs = Entities(subtype=5)
Group_Members = Users(relationship=member)
Discussions = Entities(subtype=7)
Pages = Entities(subtype=10)
Wire = Entities(subtype=17)

Content = Entities(subtype) -> entity_guid
    Elggmetadata(entity_guid) -> name_id, value_id
    Elggmetastrings(name_id OR value_id)
    
#Comments
Blog is container entity - GUID = blog guid

Blog guid = 10
search container for blog guid, return container guid
elggmetadata(container_guid)
Elggmetastrings(name_id OR value_id)

#Skills
user_GUID -> elggmetadata(container_guid) - name_id = 60

In [ ]:
# Set up subtype objects of interest

subtypes = {'blogs': 5,
            'discussions': 7,
            'pages': 10,
            'wires': 17,
            'files': 1,
            'images': 19,
            'bookmarks': 8,
            'ideas': 42
           }

subtype_list = "5 7 10 17 1 19 8 42".split()

In [ ]:
# Print list of table names

from sqlalchemy.engine import reflection

insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

## Test for Users

## Test for Blogs

In [ ]:
# Test for pulling out blog info
blogs = []

for entity, user, objects in session.query(
    Entities, Users, Objects).filter(
        Entities.subtype == 5,
        Objects.guid == Entities.guid,
        Entities.owner_guid == Users.guid,
        Users.email.contains('christopher.allison')):
    blogs.append((entity.guid, entity.subtype, user.name,
                  objects.title, objects.description))

In [ ]:
blogs[1]

## Test for running raw SQL

In [ ]:
from sqlalchemy.sql import select, text

## Pull of dept content

In [ ]:
# Take 2 - single query of DB to pull core collaborative content
# Works just fine

results = []

for e, u, o in session.query(
    Entities, Users, Objects).filter(
    Entities.owner_guid == Users.guid,
    Entities.guid == Objects.guid,
    Entities.subtype.in_(subtype_list)):
    results.append((
            e.guid, 
            e.subtype, 
            u.name,
            find_dept(u.email.lower()),
            convert_unixtime(e.time_created))
                   )

In [ ]:
len(results)

In [ ]:
# Transform raw content into dict
# Might be a better way to do this with Counter

dept_stats = {}

for d in dept_list:
    dept_stats[d] = {}
    for s in subtype_list:
        dept_stats[d][s] = 0

for r in results:
    guid, subtype, name, dept, created_date = r
    if dept in dept_list:
        dept_stats[dept][str(subtype)] += 1



In [ ]:
dept_stats['RCMP/GRC']

In [ ]:
files = 0

for d in dept_stats:
    files += dept_stats[d]['1']

print(files)
        

In [ ]:
# Transform dict indo pd.DataFrame

df = pd.DataFrame.from_dict(dept_stats, orient='index')

In [ ]:
# Add column headers

df.rename(columns = {'5' : 'Blogs', '7' : 'Discussions', '10' : 'Pages', '17' : 'Wires',
                     '8' : 'Bookmarks', '42' : 'Ideas', '19' : 'Images', '1' : 'Files'}, inplace=True)

In [ ]:
df.sort_values(by='Blogs', ascending=False ).head()

In [ ]:
df.reset_index(drop=False, inplace=True)

In [ ]:
df.rename(columns = {'index': 'Department'}, inplace=True)

In [ ]:
df = df[['Department', 'Blogs', 'Discussions', 'Pages', 'Wires',
         'Bookmarks', 'Ideas', 'Images', 'Files']]

In [ ]:
cioc = pd.merge(df, cioc_depts, how='right')

In [ ]:
cioc

In [ ]:
content = cioc[['Department', 'Blogs', 'Discussions', 'Pages', 'Wires',
         'Bookmarks', 'Ideas']]

files = cioc[['Department', 'Images', 'Files']]

In [ ]:
content.head()

In [ ]:
# Export to CSV

df.to_csv(os.path.join(output_path, 'dept_stats_{}.csv'.format(
            dt.date.today())))

## Graphing the data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')


import matplotlib
matplotlib.style.use('ggplot')

%matplotlib inline

In [ ]:
# Basic Seaborn Chart Test

sns.set_context('poster')

sns_plot = sns.barplot(palette='muted', units="Content",
    data=content.loc[df.Department == 'CANADA'])


In [ ]:
sns_plot.set_title('CANADA')

fig = sns_plot.get_figure()

fig.savefig(os.path.join(output_path, 'Canada2.png'))

In [ ]:
content.sort_values(by='Blogs', ascending=False).head()

In [ ]:
# Print all departmental graphs for content

sns.set_context('poster')

dept_list = [dept for dept in content.Department]

for dept in content.Department:
    
    sns_plot = sns.barplot(palette='muted',
    data=content.loc[content.Department == dept])
    
    sns_plot.set_title(dept)

    fig = sns_plot.get_figure()
    
    dept_title = dept.split('/')[0]

    fig.savefig(os.path.join(output_path, 'department_graphs',
                             '{}_content_2016.png'.format(dept_title)))
    
    plt.close()

In [ ]:
# Print all departmental graphs for files

sns.set_context('poster')

dept_list = [dept for dept in files.Department]

for dept in files.Department:
    
    sns_plot = sns.barplot(palette='muted',
    data=files.loc[files.Department == dept])
    
    sns_plot.set_title(dept)

    fig = sns_plot.get_figure()
    
    dept_title = dept.split('/')[0]

    fig.savefig(os.path.join(output_path, 'department_graphs',
                             '{}_files_2016.png'.format(dept_title)))
    
    plt.close()

In [ ]:
content.to_csv(os.path.join(output_path, 'gcconnex_fte_stats-{}.csv'.format(
        dt.date.today())))

files.to_csv(os.path.join(output_path, 'gcconnex_user_stats-{}.csv'.format(
        dt.date.today())))

## Pull and synthesize user stats

In [ ]:
# Select Users with Avatar & About Me completed
# name_id: 55 = About_me.description, name_id: 73 = Icontime Avatar

user_profile = []

for md, e in session.query(MetaData, Entities).filter(
    (Entities.guid == MetaData.entity_guid) &
    ((MetaData.name_id == 55) | (MetaData.name_id == 73))):
    user_profile.append((e.guid, md.name_id))

In [ ]:
user_profile[:5]

In [ ]:
user_profile_df = pd.DataFrame(user_profile, columns='User md_name'.split())

In [ ]:
def return_avatar(row):
    if row['md_name'] == 73:
        return 1
    else:
        return 0

def return_aboutme(row):
    if row['md_name'] == 55:
        return 1
    else:
        return 0

In [ ]:
user_profile_df['Avatar'] = user_profile_df.apply(
    lambda row: return_avatar (row), axis=1)

user_profile_df['Profile'] = user_profile_df.apply(
    lambda row: return_aboutme (row), axis=1)

In [ ]:
user_profile_df = user_profile_df.drop('md_name', axis=1)

In [ ]:
grouped_up = user_profile_df.groupby(['User'], sort=False).sum()


In [ ]:
grouped_up.reset_index(drop=False, inplace=True)

In [ ]:
grouped_up.head()

In [ ]:
grouped_up.describe()

## Pull User Profile Information

In [ ]:
users = []

for e, u in session.query(Entities, Users).filter(
    Entities.guid == Users.guid):
    users.append((e.guid, anonymize(bytes(u.name, 'utf-8')),
                 find_dept(u.email), 
                 convert_unixtime(e.time_created),
                  convert_unixtime(u.last_login),
                ))

In [ ]:
users[:10]

In [ ]:
user_df = pd.DataFrame(
    users, columns='User Name Department Created Last_login'.split())

In [ ]:
user_df.head()

In [ ]:
# Combine user_profiles and users into single dataframe

combined_user_df = pd.merge(grouped_up, user_df, how='right', on='User')

In [ ]:
combined_user_df.describe()

In [ ]:
combined_user_df.head()

In [ ]:
# Add zero values for users without avatars and profiles

combined_user_df['Avatar'].fillna(value=0, inplace=True)
combined_user_df['Profile'].fillna(value=0, inplace=True)

In [ ]:
combined_user_df.describe()

In [ ]:
# Generate values for users active in last 4 months

combined_user_df['Active'] = np.where(combined_user_df.Last_login > '2015-12-01', 1, 0)

In [ ]:
combined_user_df.head()

In [ ]:
mainframe = combined_user_df

In [ ]:
# Pull User Colleague Information

colleagues = []

for r in session.query(Relationships).filter(
    Relationships.relationship == 'friend'):
        colleagues.append((
            r.guid_one, r.guid_two, convert_unixtime(r.time_created)))

In [ ]:
colleagues[:10]

In [ ]:
# Create DataFrame of Colleagues

colleagues_df = pd.DataFrame(
    colleagues, columns='User friend_guid time_created'.split())

In [ ]:
# Count colleagues for each GUID

colleague_count = colleagues_df.groupby('User').count()

In [ ]:
colleague_count.reset_index(drop=False, inplace=True)

In [ ]:
colleague_count.drop('time_created', axis=1, inplace=True)
colleague_count.head()

In [ ]:
# Rename columns

colleague_count.columns = ['User', 'Colleagues']

In [ ]:
colleague_count.head()

In [ ]:
mainframe = pd.merge(mainframe, colleague_count, how='outer', on='User')

In [ ]:
# Clean Data

mainframe.fillna(value=0, inplace=True)
mainframe.drop_duplicates(inplace=True)
mainframe.describe()

In [ ]:
mainframe.tail()

In [ ]:
skills = []

for o, e in session.query(Objects, Entities).filter(
    Objects.guid == Entities.guid,
    Entities.subtype == 60):
        skills.append((
            e.owner_guid, o.title))

In [ ]:
skills_df = pd.DataFrame(skills, columns=['User', 'Skills'])

In [ ]:
skills_df.head()

In [ ]:
skills_count = skills_df.groupby('User').count()

In [ ]:
skills_count.reset_index(drop=False, inplace=True)

In [ ]:
skills_count.head()

In [ ]:
mainframe = pd.merge(mainframe, skills_count, how='outer', on='User')

In [ ]:
mainframe.head()

In [ ]:
# Pull groups by GUID

groups = []

for r in session.query(Relationships).filter(
    Relationships.relationship == 'member'):
        groups.append((
            r.guid_one, r.guid_two))

In [ ]:
groups[:5]

In [ ]:
groups_df = pd.DataFrame(groups, columns=['guid', 'group_id'])

In [ ]:
groups_count = groups_df.groupby('guid').count()

In [ ]:
groups_count.reset_index(drop=False, inplace=True)

In [ ]:
groups_count.columns = ['User', 'Groups']

In [ ]:
mainframe = pd.merge(mainframe, groups_count, how='outer', on='User')

In [ ]:
# Pull User comments
# Could also pull comments for analysis with MetaStrings as ms and ms.text

comments = []

for a, e, u in session.query(Annotations, Entities, Users).filter(
    Entities.guid == Annotations.entity_guid,
    Entities.owner_guid == Users.guid):
        comments.append((u.guid, e.guid))

In [ ]:
comments_df = pd.DataFrame(comments, columns=['User', 'entity_guid'])

In [ ]:
comment_group = comments_df.groupby('User').count()

In [ ]:
comment_group.reset_index(drop=False, inplace=True)

In [ ]:
comment_group.head()

In [ ]:
comment_group.columns = ['User', 'Comments']

In [ ]:
mainframe = pd.merge(mainframe, comment_group, how='outer', on='User')

In [ ]:
# Get rid of NaN values

mainframe.fillna(value=0, inplace=True)

In [ ]:
def threshold_check(number, threshold_value=5):
    
    if number >= threshold_value:
        return 1
    else:
        return 0

In [ ]:
mainframe['5+ Colleagues'] = mainframe['Colleagues'].apply(threshold_check)
mainframe['5+ Groups'] = mainframe['Groups'].apply(threshold_check)
mainframe['5+ Comments'] = mainframe['Comments'].apply(threshold_check)

In [ ]:
mainframe['+Skills'] = mainframe.apply(lambda x: threshold_check(x['Skills'], threshold_value=1), axis=1)

In [ ]:
mainframe.query('+Skills > 0').head()

In [ ]:
mainframe = mainframe[['User', 'Name', 'Department', 'Created', 'Last_login', 
                       'Avatar', 'Profile',
                       'Colleagues', 'Skills', 'Groups', 'Comments', 
                       'Active', '5+ Colleagues',
                       '5+ Groups', '5+ Comments', '+Skills']]

In [ ]:
mainframe.head()

In [ ]:
department_stats = mainframe[['Department', 'Active', 'Avatar', 'Profile',
                              '5+ Colleagues','5+ Groups', '5+ Comments', 
                              '+Skills']]

In [ ]:
department_stats = department_stats.groupby('Department').sum()

In [ ]:
department_stats.head()

In [ ]:
dept_members = user_df.groupby('Department').count()

In [ ]:
department_stats['Users'] = dept_members['User']

In [ ]:
department_stats = department_stats[['Users', 'Active', 'Avatar', 'Profile',
                              '5+ Colleagues','5+ Groups', '5+ Comments', 
                              '+Skills']]

In [ ]:
# Remove null values from df and remove departments with under 25 members

department_stats = department_stats.loc[department_stats.Users > 25]

In [ ]:
# Reset Index for DF

department_stats.reset_index(drop=False, inplace=True)

In [ ]:
department_stats.head()

In [ ]:
department_stats.query('Department == "RCMP/GRC"').head()

In [ ]:
cioc_depts.columns = ['Department', 'FTEs']

In [ ]:
cioc_stats = pd.merge(department_stats, cioc_depts, how='right')

In [ ]:
cioc_stats.head()

In [ ]:
ftes = cioc_stats[['Department', 'FTEs', 'Users']]
users = cioc_stats[['Department', 'Users', 'Active', 'Avatar', 'Profile', 
                    '5+ Colleagues',
                   '5+ Groups', '5+ Comments', '+Skills']]

In [ ]:
%matplotlib inline
ftes.plot()

In [ ]:
users.plot()

In [ ]:
# Save files for analysis and graphing

mainframe.to_csv(os.path.join(output_path, 'gcconnex_individual_stats-{}.csv'.format(
        dt.date.today())))

department_stats.to_csv(os.path.join(output_path, 'gcconnex_department_stats-{}.csv'.format(
        dt.date.today())))

cioc_stats.to_csv(os.path.join(output_path, 'gcconnex_cioc_stats-{}.csv'.format(
        dt.date.today())))

ftes.to_csv(os.path.join(output_path, 'gcconnex_fte_stats-{}.csv'.format(
        dt.date.today())))

users.to_csv(os.path.join(output_path, 'gcconnex_user_stats-{}.csv'.format(
        dt.date.today())))


## Graphing the data

In [ ]:
# Basic Seaborn Chart

sns_plot = sns.barplot(palette='muted', units="Users",
    data=department_stats.loc[department_stats.Department == 'CANADA'])

sns_plot.set_title('CANADA')

fig = sns_plot.get_figure()

fig.savefig('Canada.png')

In [ ]:
# Print all departmental graphs for User Info

sns.set_context('poster')

for dept in users.Department:
    
    sns_plot = sns.barplot(palette='muted',
        data=users.loc[users.Department == dept])
    
    sns_plot.set_title(dept)

    fig = sns_plot.get_figure()
    
    dept_title = dept.split('/')[0]

    fig.savefig(os.path.join(output_path, 'department_graphs',
                             '{}_users_2016.png'.format(dept_title)))
    
    plt.close()

In [ ]:
# Print all departmental graphs for FTE Info

sns.set_context('poster')

for dept in ftes.Department:
    
    sns_plot = sns.barplot(palette='muted',
        data=ftes.loc[ftes.Department == dept])
    
    sns_plot.set_title(dept)

    fig = sns_plot.get_figure()
    
    dept_title = dept.split('/')[0]

    fig.savefig(os.path.join(output_path, 'department_graphs',
                             '{}_ftes_2016.png'.format(dept_title)))
    
    plt.close()

## Network Graphing

In [ ]:
n_users = user_df.transpose()

In [ ]:
nu = n_users.to_dict()

In [ ]:
nu[0]

In [ ]:
colleagues[0]

In [ ]:
import networkx as nx

In [ ]:
G = nx.DiGraph()

In [ ]:
for user in nu:
    data = nu[user]
    G.add_node(data['User'], name=data['Name'],
               department=data['Department'], 
               created=data['Created'])

In [ ]:
for edge in colleagues:
    user, friend, connected = edge
    
    G.add_edge(user, friend, date=connected)

In [ ]:
nx.write_gexf(
    G, os.path.join(
        data_path, "gcconnex_users_content_{}.gexf".format(dt.date.today())))